In [4]:
'''
CITIES THAT WERE DELETED:
- NEW ZEALAND
- ROME
- LISBON
- IRELAND
- LOS ANGELES
- MALAGA
'''

'\nCITIES THAT WERE DELETED:\n- NEW ZEALAND\n- ROME\n- LISBON\n- IRELAND\n- LOS ANGELES\n- MALAGA\n'

In [8]:
import pandas as pd
import numpy as np

Make a new data frame in which for each listing, the availabilites are calculated for each month in the data frame

In [9]:
# read the file for the location (for now, use Amsterdam)
calendar = pd.read_csv("home_rentals/Amsterdam/calendar.csv.gz")

# make a new column which only has month-year 
calendar["date"] = pd.to_datetime(calendar["date"])
calendar["month"] = calendar["date"].dt.month
calendar["year"] = calendar["date"].dt.year
calendar['month_year'] = calendar['month'].astype(str) + '-' + calendar['year'].astype(str)

# make the new data frame
available_by_month = (calendar[calendar['available'] == 't'].groupby(['listing_id', 'month_year']).size().unstack(fill_value=0).stack().reset_index(name='days_available'))
available_by_month



,listing_id,month_year,days_available
0,2818,1-2023,0
1,2818,10-2022,8
2,2818,11-2022,6
3,2818,12-2022,0
4,2818,2-2023,0
...,...,...,...
58677,709986109543860789,6-2023,0
58678,709986109543860789,7-2023,0
58679,709986109543860789,8-2023,0
58680,709986109543860789,9-2022,20


In [10]:
# now we have to calculate a single average for each month between all the listings, so as to get an average availability 
# for a standard property in the location for each month

# Calculate the average days_available for each month_year (making sure that it rounds to full numbers)
average_by_month = (available_by_month.groupby('month_year')['days_available'].mean().round(0).reset_index(name='average_days_available'))

# convert the month_year column to datetime format 
average_by_month['month_year'] = pd.to_datetime(average_by_month['month_year'], format='%m-%Y').dt.to_period('M')

# sort the rows by ascending dates
average_by_month = average_by_month.sort_values('month_year')

# reset the row index
average_by_month.reset_index(drop=True, inplace=True)

# add the name of the location to the data frame 
average_by_month["location"] = "Amsterdam"

average_by_month



,month_year,average_days_available,location
0,2022-09,4.0,Amsterdam
1,2022-10,8.0,Amsterdam
2,2022-11,12.0,Amsterdam
3,2022-12,12.0,Amsterdam
4,2023-01,14.0,Amsterdam
5,2023-02,13.0,Amsterdam
6,2023-03,12.0,Amsterdam
7,2023-04,10.0,Amsterdam
8,2023-05,12.0,Amsterdam
9,2023-06,11.0,Amsterdam


In [34]:
# NOTE: due to the use of both Macbooks and Windows computers in the project, the file path uses either / or \ in the file path
# as such, we have included a boolean to change the file path such that it works for the user's computer

macbook = True

import pandas as pd 

# now make a data frame with the average number of rentals per month per location. 
# In order to do this, have to use the same code as above for each city 

# Make a function such that it's easier to use for all locations
def avg_availability_location(location_name):
    # read the file for the location (for now, use Amsterdam)
    if macbook == False:
        location_path = "home_rentals" + location_name + "\calendar.csv.gz"
    if macbook == True:
        location_path = "home_rentals" + location_name + "/calendar.csv.gz"

    calendar = pd.read_csv(location_path)

    # make a new column which only has month-year 
    calendar["date"] = pd.to_datetime(calendar["date"])
    calendar["month"] = calendar["date"].dt.month
    calendar["year"] = calendar["date"].dt.year
    calendar['month_year'] = calendar['month'].astype(str) + '-' + calendar['year'].astype(str)

    # make the new data frame
    available_by_month = (calendar[calendar['available'] == 't'].groupby(['listing_id', 'month_year']).size().unstack(fill_value=0).stack().reset_index(name='days_available'))

    # Calculate the average days_available for each month_year (making sure that it rounds to full numbers)
    average_by_month = (available_by_month.groupby('month_year')['days_available'].mean().round(0).reset_index(name='average_days_available'))

    # convert the month_year column to datetime format 
    average_by_month['month_year'] = pd.to_datetime(average_by_month['month_year'], format='%m-%Y').dt.to_period('M')

    # sort the rows by ascending dates
    average_by_month = average_by_month.sort_values('month_year')

    # reset the row index
    average_by_month.reset_index(drop=True, inplace=True)

    # add the name of the location to the data frame 
    average_by_month["location"] = location_name[1:]

    return average_by_month



# run the function for each location, appending the location's df to an empty one for all of the locations

all_locations = [r"\Amsterdam", r"\Barcelona", r"\Euskadi", r"\Geneva", r"\Madrid", r"\Mallorca", r"\Malta", r"\Menorca", r"\Mexico City", r"\Milan", r"\Naples", 
                 r"\Rotterdam", r"\San Diego", r"\San Francisco", r"\Tokyo", r"\Toronto", r"\Valencia", r"\Vancouver", r"\Vienna", r"\Washington", r"\Winnipeg", r"\Zurich"]

if macbook == True:
    for location in all_locations:
        # Convert the string to a list of characters
        str_location = list(location)
        str_location[0] = "/"
        # Join the list back into a string
        location = ''.join(str_location)


available_all_loc = pd.DataFrame()

# Create a list to store DataFrames for each location
dfs_list = []

for location in all_locations:
    average_by_month = avg_availability_location(location)
    dfs_list.append(average_by_month)


# Concatenate all DataFrames in the list
available_all_loc = pd.concat(dfs_list, ignore_index=True)

# visualise the final data frame
available_all_loc




KeyboardInterrupt: 

In [32]:
import numpy as np

# check that there is the same number of months per location 

all_locations = ["Amsterdam", "Barcelona", "Euskadi", "Geneva", "Madrid", "Mallorca", "Malta", "Menorca", "Mexico City", "Milan", "Naples", 
                 "Rotterdam", "San Diego", "San Francisco", "Tokyo", "Toronto", "Valencia", "Vancouver", "Vienna", "Washington", "Winnipeg", "Zurich"]

occurences_per_loc = np.zeros((len(all_locations),1))
counter = 0

for location in all_locations:
    value = available_all_loc[available_all_loc["location"] == location].count()
    #occurences_per_loc[counter,0] = location
    occurences_per_loc[counter,0] = value[0]
    counter += 1

print(occurences_per_loc)

[[13.]
 [13.]
 [14.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]]


C:\Users\elisa\AppData\Local\Temp\ipykernel_9688\2006174024.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  occurences_per_loc[counter,0] = value[0]
C:\Users\elisa\AppData\Local\Temp\ipykernel_9688\2006174024.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  occurences_per_loc[counter,0] = value[0]
C:\Users\elisa\AppData\Local\Temp\ipykernel_9688\2006174024.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  occurences_per_

In [ ]:
''' 

TO DO:

- split the data set into test and train 
- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september
    then it's skewed data because it'll say it was either completely free/completely booked for that month)


'''

" \n\nTO DO:\n\n- split the data set into test and train \n- check when the data starts and ends --> if we don't have full months then it won't be valid (e.g. if it starts towards the end of september\n    then it's skewed data because it'll say it was either completely free/completely booked for that month)\n\n\n"

## Data Anaysis


In [ ]:

city = available_all_loc['location'].unique()

NameError: name 'available_all_loc' is not defined